In [2]:
import tkinter as tk
from tkinter import messagebox

def check_length(event):
    user_input = entry.get()
    if len(user_input) != 10:
        messagebox.showwarning("Warning", "Input must be 10 digits.")

root = tk.Tk()
root.title("Input Length Checker")

label = tk.Label(root, text="Enter 10-digit input:")
label.pack(pady=5)

entry = tk.Entry(root)
entry2 = tk.Entry(root)
entry.pack(pady=5)
entry2.pack(pady=10)
entry.bind("<FocusOut>", check_length)  # Bind event listener to check length when focus leaves the entry field

root.mainloop()


In [11]:
import sqlite3
import pandas as pd
#import pandas as pd
import gspread as gs
from google.oauth2 import service_account
import time
import datetime
#from gspread_pandas import Spread, Client
start_time = time.time()
SCOPES = [
'https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive'
]
SERVICE_ACCOUNT_FILE = r'C:\Users\KP\Development\IMS_2024\main_ttk\sheets-to-python-credential.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

client = gs.authorize(credentials)

Spread = client.open("OP Register")


pharmacyWS = Spread.worksheet("Pharmacy")
def pharmData():
        pwsFirstRow = pharmacyWS.row_values(1)
        #print(pwsFirstRow)
        pwsBillNoColNo = pwsFirstRow.index("Bill No")+1
        pwsMedNameColNo = pwsFirstRow.index("Medicine name")+1
        pwsDateColNo = pwsFirstRow.index("Date")+1
        pwsPatientNameColNo = pwsFirstRow.index("Name")+1
        pwsQtyColNo = pwsFirstRow.index("Quantity")+1
        pwsLastRowNo = len(pharmacyWS.col_values(pwsMedNameColNo))+1
        #print(pwsLastRowNo, pwsBillNoColNo, pwsMedNameColNo, pwsDateColNo, pwsQtyColNo, pwsPatientNameColNo)
        return pwsLastRowNo, pwsBillNoColNo, pwsMedNameColNo, pwsDateColNo, pwsQtyColNo, pwsPatientNameColNo

"""start_time = time.time()

pharmData()
end_time = time.time()

print(end_time-start_time)"""

medListWS = Spread.worksheet("Medicine List")


def getMedData():
        mlsMedNameColNo = medListWS.find("Name", in_row=1).col
        medList = medListWS.col_values(mlsMedNameColNo)
        medList = medList[2:]
        medList.sort(key=str.lower)
        return medList








list_of_lists = medListWS.get_all_values()
medListData = pd.DataFrame(list_of_lists)
medListData.columns = medListData.iloc[0]
medListData = medListData[1:]
#print(medListData)



def getInvoiceData():
        insFirstRow = inoviceWS.row_values(1)
        insinvNoColNo= insFirstRow.index("Inovice No")+1
        insPatientNameColNo= insFirstRow.index("Name")+1
        insBillAmountColNo= insFirstRow.index("Bill Amount")+1
        insDiscountColNo= insFirstRow.index("Discount")+1
        insPayModeColNo= insFirstRow.index("Payment Mode")+1
        insCashColNo= insFirstRow.index("Cash")+1
        insUPIColNo= insFirstRow.index("UPI")+1
        insDateColNo = insFirstRow.index("Date")+1
        insLastRowNo = len(inoviceWS.col_values(insDateColNo))+1
        return insLastRowNo, insPatientNameColNo, insDateColNo, insBillAmountColNo, insPayModeColNo, insDiscountColNo, insCashColNo, insUPIColNo, insinvNoColNo


inoviceWS = Spread.worksheet("Invoices")

opWS =Spread.worksheet("OP")

def getOPData():
        oPFirstRow = opWS.row_values(1)
        oPUIDColNo = oPFirstRow.index("UID")+1
        oPDateColNo = oPFirstRow.index("Date")+1
        oPNameColNo = oPFirstRow.index("Name")+1
        oPPhoneColNo = oPFirstRow.index("Phone No")+1
        oPGenderColNo = oPFirstRow.index("Gender")+1
        oPAgeColNo = oPFirstRow.index("Age")+1
        oPPayModeColNo = oPFirstRow.index("Payment mode")+1
        oPAmountColNo = oPFirstRow.index("Amount")+1
        opLastRow = len(opWS.col_values(oPNameColNo))

        return oPUIDColNo, oPDateColNo, oPNameColNo, oPPhoneColNo, oPPayModeColNo, oPAmountColNo, opLastRow, oPGenderColNo, oPAgeColNo


aCountWS = Spread.worksheet("ACount")

def getClientid(name):
    name = name.strip()
    name = name.replace(" ", "")
    name = name.replace(".", "")
    now = datetime.datetime.now()  
    year_str = str(now.year)[-2:]  
    month_str = str(now.month).zfill(2)
    name_prefix = name[:3].upper()
    first_letter = name_prefix[0]   
    
    currLetter = aCountWS.find(first_letter,in_column=1).row
    currCount = int(aCountWS.cell(currLetter,2).value)

    
    count = currCount + 1
    serial_num = str(count).zfill(2)
    
    return f"{year_str}{month_str}{name_prefix}{serial_num}"





def getBillNo():
        insDateColNo = inoviceWS.find("Date", in_row=1).col
        insinvNoColNo = inoviceWS.find("Inovice No", in_row=1).col
        insLastRowNo = len(inoviceWS.col_values(insDateColNo))

        currBillNo = inoviceWS.cell(insLastRowNo, insinvNoColNo).value

        return currBillNo

end_time = time.time()

print(end_time-start_time)



4.556128740310669


In [4]:
for x in range (2,9):
    name = opWS.cell(x,10).value
    cid = getClientid(name)
    opWS.update_cell(x,3,name)
    opWS.update_cell(x,1,cid)
    opWS.update_cell(x,11,"")
    time.sleep(1)

In [9]:
#def getCurrentDayPatients():
from datetime import date

today = date.today().strftime("%d-%b")
columnheaders = opWS.row_values(1)
oPUIDColNo, oPDateColNo, oPNameColNo, oPPhoneColNo, oPPayModeColNo, oPAmountColNo, opLastRow, oPGenderColNo, oPAgeColNo = getOPData()
rowsWithDate = [opWS.row_values(x.row) for x in opWS.findall(today, in_column=oPDateColNo)]
currentDayPatients = pd.DataFrame(rowsWithDate)
print(columnheaders)
currentDayPatients
currentDayPatients.columns = columnheaders
currentDayPatients    
    #$return currentDayPatients

#getCurrentDayPatients()



['UID', 'Date', 'Name', 'Phone No', 'Gender', 'Age', 'Procedure', 'Payment mode', 'Amount', 'Comments']


,UID,Date,Name,Phone No,Gender,Age,Procedure,Payment mode,Amount,Comments
0,2402HUS96,27-Feb,S.Saraswati Amma,9963148667,,,NO,,NO,None
1,2402HUS97,27-Feb,S.Jayala Lakshmi,9441088440,,,NO,,NO,None
2,2402HUS98,27-Feb,Asma,8074640549,,,con,Cash,200,None
3,2402HUS99,27-Feb,Sartaj,9885385417,,,con,Cash,200,CLEAR
4,2402HUS100,27-Feb,Aryan,9154762075,,,con,Cash,200,None


In [17]:
insLastRowNo, insPatientNameColNo, insDateColNo, insBillAmountColNo, insPayModeColNo, insDiscountColNo, insCashColNo, insUPIColNo, insinvNoColNo = getInvoiceData()
pwsLastRowNo, pwsBillNoColNo, pwsMedNameColNo, pwsDateColNo, pwsQtyColNo, pwsPatientNameColNo = pharmData()
def getBillDetails():
    insLastRowNo = len(inoviceWS.col_values(insDateColNo))
    #print(insLastRowNo,insinvNoColNo)
    billNo = inoviceWS.cell(insLastRowNo, insinvNoColNo).value
    rowsWithBillNo = [pharmacyWS.row_values(x.row) for x in pharmacyWS.findall(billNo, in_column=pwsBillNoColNo)]
    return rowsWithBillNo, billNo
    
billData, billNo = getBillDetails()
billData[0]

8 1


['PM2406007',
 '1-Mar',
 'Sangeetha',
 'Finecez M',
 '20',
 'Tablet',
 '',
 '10.5',
 '210',
 '',
 '7',
 '453527',
 '',
 '',
 '',
 '',
 '86']

In [16]:
from reportlab.lib.units import inch
from datetime import date

def my_temp(c):
    # Set A5 size dimensions
    a5_width, a5_height = 5.8 * inch, 8.3 * inch
    c.translate(a5_width - 0.05 * inch, 0.05 * inch)  # Adjust origin to top-left corner
    
    # define a smaller font for A5
    c.setFont("Helvetica", 10)
    
    # Other adjustments...
    c.setStrokeColorRGB(0, 1, 0)
    c.setFillColorRGB(0,0,1)
    c.rotate(90)
    #c.drawImage('C:\\Users\\KP\\Development\\IMS_2024\\main_ttk\\logoMain.png', 3.3 * inch, 3.0 * inch)
    c.setFont("Helvetica", 20)
    c.drawString(0.2*inch, 5.4 * inch, "Pranith Medical Store")
    c.setFont("Helvetica", 12)
    c.drawString(0.2*inch, 5.2 * inch, "#25-684-15, TTD Road, Doctor's Lane")
    c.drawString(0.2*inch, 5.0 * inch, "Nandyal, 518501")
    c.setStrokeColorRGB(0, 1, 0)  # line colour
    c.line(0.2, 4.9 * inch, 8.0 * inch, 4.9 * inch)
    
    
    dt = date.today().strftime('%d-%b-%Y')
    c.drawString(6.8 * inch, 5.25 * inch, dt)
    
    
    # Other adjustments...
    
    c.setFillColorRGB(1, 0, 0)  # font colour
    c.setFont("Times-Bold", 16)
    c.drawString(3.5 * inch, 5.0 * inch, 'INVOICE')

    
    # Other adjustments...
    
    # Set vertical line color
    c.setStrokeColorCMYK(0, 0, 0, 1)
    
    # Other adjustments...
    
    # Set horizontal line total
    global ProductLine
    global RateLine
    global QtyLine
    global AmtLine
    global billYLine
    global PcodeLine

    PcodeLine = 0.5
    ProductLine = PcodeLine + 0.9
    RateLine = ProductLine + 2.4
    QtyLine = RateLine+0.7
    AmtLine = QtyLine+1.2
    
    billYLine = 1.9
    
    c.setFont("Times-Bold", 12)
    c.drawString(0.2 * inch, 4.7 * inch, 'Id')
    c.line(PcodeLine * inch,4.9* inch, PcodeLine * inch, billYLine * inch)    
    c.drawString((PcodeLine+0.05) * inch, 4.7 * inch, 'P Code')
    c.line(ProductLine * inch,4.9* inch, ProductLine * inch, billYLine * inch)
    c.drawString((ProductLine+0.05) * inch, 4.7 * inch, 'Product')  
    c.line(RateLine * inch,4.9* inch, RateLine * inch, billYLine * inch)
    c.drawString((RateLine+0.05) * inch, 4.7 * inch, 'Rate')
    c.line(QtyLine * inch,4.9* inch, QtyLine * inch, billYLine * inch)
    c.drawString((QtyLine +0.05) * inch, 4.7 * inch, 'Quantity')
    c.line(AmtLine * inch,4.9* inch, AmtLine * inch, billYLine * inch)
    c.drawString((AmtLine+.05) * inch, 4.7 * inch, 'Amount')
    c.line(0.2, 4.6 * inch, 8.0 * inch, 4.6 * inch)
    # Other adjustments...
    
    
    c.drawString(0.5 * inch, -5.5 * inch, 'Discount')
    c.drawString(0.5 * inch, -6.2 * inch, 'Tax')
    
    # Other adjustments...
    
    c.setFont("Times-Bold", 12)
    c.drawString(1 * inch, -6.7 * inch, 'Total')
    
    # Other adjustments...
    
    c.setFont("Times-Roman", 12)
    c.drawString(3.5 * inch, -7.3 * inch, 'Signature')
    
    # Bottom Line color
    c.setStrokeColorRGB(0.1, 0.8, 0.1)
    c.line(0, -7.8 * inch, 5.5 * inch, -7.8 * inch)
    
    c.setFont("Helvetica", 6)  # font size
    c.setFillColorRGB(1, 0, 0)  # font colour
    c.drawString(0, -8 * inch, u"\u00A9" + " plus2net.com")
    
    return c


from reportlab.pdfgen import canvas
#$bill_No = "PM2403404"

from reportlab.lib.units import inch
from reportlab.lib.pagesizes import letter, A5
#from temp_invoice import my_temp # import the template
#from invoice_data import *  # get all data required for invoice
billData, billNo=getBillDetails()
def printBill(my_prod, billNo):
    my_path=r'C:\Users\KP\Development\IMS_2024\main_ttk\Invoices\{}.pdf'.format(billNo) 
    c = canvas.Canvas(my_path,pagesize=A5)
    
    c=my_temp(c) # run the template
    
    print(ProductLine, QtyLine, AmtLine, RateLine)
    c.setFillColorRGB(0,0,0) # font colour

    c.setFont("Helvetica", 12)
    c.drawString(6.8 * inch, 4.95 * inch, billNo)
    c.setFont("Helvetica", 8)
    row_gap=0.15 # gap between each row
    line_y=4.3 # location of fist Y position 
    total=0
    i = 1
    ['PM2406007', '1-Mar', 'Sangeetha', 'Finecez M', '20', 'Tablet', '', '10.5',
 '210', '', '7', '453527', '', '', '', '', '86']
    for rec in my_prod:
        c.drawString((0.2+0.05)*inch,line_y*inch,str(i))
        c.drawString((PcodeLine+0.05)*inch,line_y*inch,str(rec[5])) # product Code
        c.drawString((ProductLine+0.05)*inch,line_y*inch,str(rec[3])) # p Name
        c.drawString((RateLine +0.05)*inch,line_y*inch,str(rec[7])) # p Price
        c.drawString((QtyLine+0.05)*inch,line_y*inch,str(rec[4])) # p Qant 
        sub_total=float(rec[7])*float(rec[4])
        c.drawString((AmtLine+0.05)*inch,line_y*inch,str(sub_total)) # Sub Total 
        total=round(total+sub_total,1)
        line_y=line_y-row_gap
        i+=1
    c.drawRightString((AmtLine -0.05)*inch, (billYLine-row_gap) * inch, 'Bill Amount')
    c.drawString((AmtLine+0.05)*inch,(billYLine-row_gap)*inch,str(float(total))) # Total 
    #discount=round((discount_rate/100) * total,1)
    #c.drawRightString(4*inch,1.8*inch,str(discount_rate)+'%') # discount
    #c.drawRightString(7*inch,1.8*inch,'-'+str(discount)) # discount
    #tax=round((tax_rate/100) * (total-discount),1)
    #c.drawRightString(4*inch,1.2*inch,str(tax_rate)+'%') # tax 
    #c.drawRightString(7*inch,1.2*inch,str(tax)) # tax 
    total_final=total
    c.setFont("Times-Bold", 22)
    c.setFillColorRGB(1,0,0) # font colour
    #c.drawRightString(2*inch,0.8*inch,str(total_final)) # tax 
    c.rotate(90)
    c.showPage()
    c.save()


printBill(billData, billNo)

8 1
1.4 4.5 5.7 3.8


TypeError: can't multiply sequence by non-int of type 'str'